In [63]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from collections import Counter


# Step 1: Read your dataset
df = pd.read_csv("data/1_filtered_us_artists_2024_2025.csv")  # or pd.read_excel() if using Excel


In [64]:
df['snapshot_date'] = pd.to_datetime(df['snapshot_date'])

# 3. Compute the total number of unique days in the dataset (for normalization)
total_days = df['snapshot_date'].nunique()

# 4. Extract primary_artist by splitting the 'artists' column on the comma and taking the first value
df['primary_artist'] = df['artists'].apply(lambda x: x.split(',')[0].strip())

# 5. Compute aggregated track-level metrics for each unique song (spotify_id)
#    - avg_daily_rank: mean of daily_rank values
#    - avg_popularity: mean of popularity values
#    - days_in_top50: count of days the song appears in the Top 50
#    - primary_artist: first occurrence (used later for artist info, if needed)
track_metrics = df.groupby('spotify_id').agg(
    avg_daily_rank=('daily_rank', 'mean'),
    avg_popularity=('popularity', 'mean'),
    days_in_top50=('snapshot_date', 'count'),
    primary_artist=('primary_artist', 'first')
).reset_index()

# 6. Compute the normalized base index for each track (0-1 scale)
#    Formula:
#    popularity_index = (days_in_top50 / total_days) * ((51 - avg_daily_rank) / 50) * (avg_popularity / 100)
track_metrics['popularity_index'] = (
    (track_metrics['days_in_top50'] / total_days) *
    ((51 - track_metrics['avg_daily_rank']) / 50) *
    (track_metrics['avg_popularity'] / 100)
)

# 7. To preserve all additional columns, take the first occurrence of each unique song from the original dataset
first_occurrence = df.drop_duplicates(subset=['spotify_id']).copy()

# 8. Merge the computed popularity_index back into the first_occurrence DataFrame
final_df = first_occurrence.merge(
    track_metrics[['spotify_id', 'popularity_index']],
    on='spotify_id',
    how='left'
)

# 9. Save the final DataFrame to CSV
final_df.to_csv("us_popularity_index.csv", index=False)


This added popularity index which is calculated based 
popularity_index = 0.5 * avg_popularity 
                 + 0.3 * mean_inv_rank 
                 + 0.2 * days_in_top50

Why this formula?

avg_popularity captures fan engagement on Spotify (0–100 scale)

mean_inv_rank reflects chart strength (inverted rank → higher = better)

days_in_top50 represents chart longevity (how long it stayed relevant)

We weight popularity most (50%) because it best reflects streaming success, but we still value chart rank and consistency.